# Import Libraries

In [1]:
#import mediapipe
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
import pickle

#import from sklearn
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

#utility that draws the whole connection
mp_drawings = mp.solutions.drawing_utils

#mediapipe solutions. landmarks from holistic
mp_holistic = mp.solutions.holistic

# Make Detections with Model

In [2]:
#import the pickled gb model
#rb = read b
with open('../Capstone/data/workout_classifier.pkl', 'rb') as f:
    workout_model = pickle.load(f)

In [3]:
workout_model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) #first point of the body
    b = np.array(b) #second point of the body
    c = np.array(c) #third point of the body
    
    #equation to get the radian
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    #equation to find the absolute angle
    angle = np.abs(radians*180.0/np.pi)
    
    #if the poser calcuate the angle to be greater than 180, that means that it is miscalculting
    if angle > 180:
        angle = 360-angle
        
    return angle

In [5]:
#create a position finder function
def findPosition(image, draw=True):
    lmList=[]
    if results.pose_landmarks:
        mp_drawings.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = image.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            lmList.append([id, cx, cy])
    return lmList

In [9]:
#Video Feed

# Setting up video capture device
# 0 = built-in webcam & 2 = External webcam
# If want to grab a video clip, type the name of video and its file loc within (...)
cap = cv2.VideoCapture(2)


# counter and stage variables
left_bi_curl_counter = 0
left_bi_curl_stage = None

right_bi_curl_counter = 0
right_bi_curl_stage = None

shoulder_press_counter = 0
shoulder_press_stage = None

squat_counter = 0
squat_stage = None


# Start the holistic model
# 'Holistic' allows me to access the Holistic estimation model
# '0.6' = 60%
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    
    #While loop to allow video to stay open till I give command to close window
    while cap.isOpened():
        #frame is the frame that will be returning
        #cap.read is where the webcam will capture and read the video
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #apply and set the image to be unwriteable
        image.flags.writeable = False
        

        #Make Detections
        results = holistic.process(image)

#         #To show the coords of each types of landmarks (face, pose, hand), add '.face_landmarks' at end of "results" variable
#         print(results)

        #setting image writeable back to true to be able process it
        image.flags.writeable = True

        #recoloring it back to BGR b/c it will rerender back to opencv
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lmList = findPosition(image, draw=True)
        
        #EXTRACT LANDMARKS
        #create a new variable "landmarks" that will display the landmarks on the body when the feed is live
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get coordinates
            left_hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
            left_knee = [landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_holistic.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_ANKLE.value].y]
            left_heel = [landmarks[mp_holistic.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_hip = [landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
            right_knee = [landmarks[mp_holistic.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_holistic.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_ANKLE.value].y]
            right_heel = [landmarks[mp_holistic.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            #calculate angle 
            left_bicep_angle = round(calculate_angle(left_shoulder, left_elbow, left_wrist), 1)
            left_arm_raise_angle = round(calculate_angle(left_hip, left_shoulder, left_elbow), 1)
            left_side_bend_angle = round(calculate_angle(left_shoulder, left_hip, left_knee), 1)
            left_knee_angle = round(calculate_angle(left_hip, left_knee, left_ankle), 1)
            left_ankle_angle = round(calculate_angle(left_knee, left_ankle, left_foot_index), 1)
            
            right_bicep_angle = round(calculate_angle(right_shoulder, right_elbow, right_wrist), 1)
            right_arm_raise_angle = round(calculate_angle(right_hip, right_shoulder, right_elbow), 1)
            right_side_bend_angle = round(calculate_angle(right_shoulder, right_hip, right_knee), 1)
            right_knee_angle = round(calculate_angle(right_hip, right_knee, right_ankle), 1)
            right_ankle_angle = round(calculate_angle(right_knee, right_ankle, right_foot_index), 1)
            
            #create visual markers of the angle
            #colour coord is in GRB instead of RGB
            cv2.putText(image, str(left_bicep_angle),
                        tuple(np.multiply(left_elbow, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(left_arm_raise_angle),
                        tuple(np.multiply(left_shoulder, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(left_side_bend_angle),
                        tuple(np.multiply(left_hip, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(left_knee_angle),
                        tuple(np.multiply(left_knee, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(left_ankle_angle),
                        tuple(np.multiply(left_ankle, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2, cv2.LINE_AA
                       )
            
            
            cv2.putText(image, str(right_bicep_angle),
                        tuple(np.multiply(right_elbow, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(right_arm_raise_angle),
                        tuple(np.multiply(right_shoulder, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(right_side_bend_angle),
                        tuple(np.multiply(right_hip, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(right_knee_angle),
                        tuple(np.multiply(right_knee, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(right_ankle_angle),
                        tuple(np.multiply(right_ankle, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            
#             print(landmarks)
        #if it can't detect the body, it will just continue to play video feed
        except:
            pass

        #Draw the face landmarks
        mp_drawings.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawings.DrawingSpec(color=(0,155,0), thickness=1, circle_radius=1),
                                  mp_drawings.DrawingSpec(color=(100,155,0), thickness=1, circle_radius=1))
        
        
#         #Draw the right hand landmarks
#         mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawings.DrawingSpec(color=(0,155,155), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(100,155,0), thickness=2, circle_radius=1))
        
#         #Draw the left hand landmarks
#         mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawings.DrawingSpec(color=(200,155,50), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(0,255,88), thickness=2, circle_radius=1))
        
        #Draw the pose landmarks
        mp_drawings.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawings.DrawingSpec(color=(255,155,255), thickness=1, circle_radius=1),
                                  mp_drawings.DrawingSpec(color=(100,155,200), thickness=2, circle_radius=1))
        
        #export coordinates
        try:
            #get all the coordinates from the landmarks
            pose_workout = results.pose_landmarks.landmark
            
            #extracting all the coords and converting it to an array
            pose_workout_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_workout]).flatten()
            
            #covert the array to a list
            pose_workout_row = list(pose_workout_row)
            
            #concatenate rows together
            #when there are more than just one
            total_row = pose_workout_row
            
#             #append class name
#             total_row.insert(0, workout_name)
            
#             #create a new CSV writer for pose landmarks
#             #mode='a' as append
#             with open('../Capstone/data/workout_pose_coord_loc.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(total_row)
            
    
            #make workout pose detection
            x = pd.DataFrame([total_row]) #crate a dataframe
            body_lang_class = workout_model.predict(x)[0] 
            body_lang_prob = workout_model.predict_proba(x)[0] #predict the probability of the landmark location
            print(body_lang_class, body_lang_prob) #print the two key components for the prediction
            
            # Use left ear coords to place the workout box
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x,
            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y)), [640,480]).astype(int))
            
            #size and position of the workout box
            cv2.rectangle(image,(coords[0], coords[1]+5), #position of the box near the left ear
                          (coords[0]+len(body_lang_class)*20, coords[1]-30), #position of the box on the top left corner
                          (200, 200, 200), -1) #color of the box and fill the color within the box
            
            #font size, format and style
            cv2.putText(image, body_lang_class, coords,cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #position of the status box
            cv2.rectangle(image, (240,0), (640, 60), (245, 117, 16), -1)
            
            # Display Workout
            cv2.putText(image, 'Workout', (345,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, body_lang_class.split(' ')[0], (350,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'Prob', (245,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_lang_prob[np.argmax(body_lang_prob)],1)*100), 
                        (240,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Produce a Pop up counter when the body_lang_class reads the correct manoeuver
            #call out left bicep curl
            if body_lang_class == 'Left_Bicep_Curl':
                #draw the pose annotation on the image
                #marking the elbow's circle bigger
                #left elbow
                cv2.circle(image, (lmList[13][1], lmList[13][2]), 20, (0,255,255), cv2.FILLED)
                
                #curl counter logic for left bicep curl
                if left_bicep_angle > 160:
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 10, (0,0,255), cv2.FILLED)
                    left_bi_curl_stage = "DOWN"
                    
                if left_bicep_angle > 140 and left_bicep_angle <= 160:
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 12, (0,50,200), cv2.FILLED)
                    left_bi_curl_stage = "Down Mid"
                    
                if left_bicep_angle > 110 and left_bicep_angle <= 140:
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 14, (0,100,150), cv2.FILLED)
                    left_bi_curl_stage = "Mid Down"
                    
                if left_bicep_angle > 80 and left_bicep_angle <= 110:
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 16, (0,150,100), cv2.FILLED)
                    left_bi_curl_stage = "Mid Up"
                    
                if left_bicep_angle > 50 and left_bicep_angle <= 80:
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 18, (0,200,50), cv2.FILLED)
                    left_bi_curl_stage = "Up Mid"
                    
                if left_bicep_angle > 30 and left_bicep_angle <= 50:
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 18, (0,225,30), cv2.FILLED)
                    left_bi_curl_stage = "Almost There"
                    
                if left_bicep_angle <= 25 and left_bi_curl_stage == "Almost There":
                    left_bi_curl_stage = "UP"
                    cv2.circle(image, (lmList[13][1], lmList[13][2]), 20, (0,255,0), cv2.FILLED)
                    left_bi_curl_counter +=1
                    print(f"Current Left Bicep Curl Rep: {left_bi_curl_counter}")
                    
                #add curl counter
                #setup a status box
                cv2.rectangle(image, (0,0), (300,60), (24,117,16), -1)  #(0,0) =  start point of rectangle
                                                                        #(225,73) = end point of rectangle
                                                                        #(24,117,16) = RGB
                                                                        #-1 = fill the box with color
                #rep data(left arm)
                cv2.putText(image, 'LT REPS: ', (15,25),                        #(15,12) = start coord
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1, cv2.LINE_AA) #font type = hersey
                                                                                        #text size = 0.5
                                                                                        #text color = 255,255,0
                                                                                        #line thickness = 1
                                                                                        #line type = AA
                cv2.putText(image, str(left_bi_curl_counter), (10,60), #(10,60) = start coord
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) #font type = hersey
                                                                                        #text size = 0.5
                                                                                        #text color = 255,255,255
                                                                                        #line thickness = 2
                                                                                        #line type = AA
                #rep position(left arm)
                cv2.putText(image, 'LT ARM POS: ', (165,25),                  #(165,12) = start coord
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA) #font type = hersey
                                                                                      #text size = 0.5
                                                                                      #text color = 0,255,0
                                                                                      #line thickness = 1
                                                                                      #line type = AA
                cv2.putText(image, left_bi_curl_stage, (160,60),                         #(160,60) = start coord
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) #font type = hersey
                                                                                        #text size = 1
                                                                                        #text color = 255,255,255
                                                                                        #line thickness = 2
                                                                                        #line type = AA
            #call out right bicep curl
            if body_lang_class == 'Right_Bicep_Curl':
                #draw the pose annotation on the image
                #marking the elbow's circle bigger
                #right elbow
                cv2.circle(image, (lmList[14][1], lmList[14][2]), 20, (255,255,0), cv2.FILLED)
                
                #curl counter logic for right bicep curl
                if right_bicep_angle > 160:
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 10, (255,0,255), cv2.FILLED)
                    right_bi_curl_stage = "DOWN"
                    
                if right_bicep_angle > 140 and right_bicep_angle <= 160:
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 12, (200,50,200), cv2.FILLED)
                    right_bi_curl_stage = "Down Mid"
                    
                if right_bicep_angle > 110 and right_bicep_angle <= 140:
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 14, (150,100,150), cv2.FILLED)
                    right_bi_curl_stage = "Mid Down"
                    
                if right_bicep_angle > 80 and right_bicep_angle <= 110:
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 16, (100,150,100), cv2.FILLED)
                    right_bi_curl_stage = "Mid"
                    
                if right_bicep_angle > 50 and right_bicep_angle <= 80:
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 18, (50,200,50), cv2.FILLED)
                    right_bi_curl_stage = "Mid Up"
                    
                if right_bicep_angle > 30 and right_bicep_angle <= 50:
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 18, (300,225,30), cv2.FILLED)
                    right_bi_curl_stage = "Almost There"
                    
                if right_bicep_angle <= 25 and right_bi_curl_stage == "Almost There":
                    right_bi_curl_stage = "UP"
                    cv2.circle(image, (lmList[14][1], lmList[14][2]), 20, (0,255,0), cv2.FILLED)
                    right_bi_curl_counter +=1
                    print(f"Current Right Bicep Curl Rep: {right_bi_curl_counter}")
                    
                #add curl counter
                #setup a status box
                cv2.rectangle(image, (0,0), (300,60), (24,117,16), -1)  #(0,0) =  start point of rectangle
                                                                        #(225,73) = end point of rectangle
                                                                        #(24,117,16) = RGB
                                                                        #-1 = fill the box with color
                #rep data(left arm)
                cv2.putText(image, 'RT REPS: ', (15,25),                        #(15,12) = start coord
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,150), 1, cv2.LINE_AA) #font type = hersey
                                                                                        #text size = 0.5
                                                                                        #text color = 255,255,0
                                                                                        #line thickness = 1
                                                                                        #line type = AA
                cv2.putText(image, str(right_bi_curl_counter), (10,60), #(10,60) = start coord
                            cv2.FONT_HERSHEY_COMPLEX, 1, (200,0,255), 2, cv2.LINE_AA) #font type = hersey
                                                                                        #text size = 0.5
                                                                                        #text color = 255,255,255
                                                                                        #line thickness = 2
                                                                                        #line type = AA
                #rep position(left arm)
                cv2.putText(image, 'RT ARM POS: ', (165,25),                  #(165,12) = start coord
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (125,255,125), 1, cv2.LINE_AA) #font type = hersey
                                                                                      #text size = 0.5
                                                                                      #text color = 0,255,0
                                                                                      #line thickness = 1
                                                                                      #line type = AA
                cv2.putText(image, right_bi_curl_stage, (160,60),                         #(160,60) = start coord
                            cv2.FONT_HERSHEY_COMPLEX, 1, (185,185,34), 2, cv2.LINE_AA) #font type = hersey
                                                                                        #text size = 1
                                                                                        #text color = 255,255,255
                                                                                        #line thickness = 2
                                                                                        #line type = AA    
                #call out left bicep curl
                if body_lang_class == 'Shoulder_Press':
                    #draw the pose annotation on the image
                    #marking the shoulders circle bigger
                    #shoulders
                    cv2.circle(image, (lmList[12][1], lmList[12][2]), 20, (0,255,255), cv2.FILLED)
                    cv2.circle(image, (lmList[11][1], lmList[11][2]), 20, (0,255,255), cv2.FILLED)

                    #press counter logic for shoulder press
                    if left_shoulder > 65 and right_shoulder > 65 and left_elbow >=85 and right_elbow >= 85:
                        cv2.circle(image, (lmList[12][1], lmList[12][2]), 10, (0,0,255), cv2.FILLED)
                        cv2.circle(image, (lmList[11][1], lmList[11][1]), 10, (0,0,255), cv2.FILLED)
                        shoulder_press_stage = "DOWN"

#                     if left_bicep_angle > 140 and left_bicep_angle <= 160:
#                         cv2.circle(image, (lmList[13][1], lmList[13][2]), 12, (0,50,200), cv2.FILLED)
#                         left_bi_curl_stage = "Down Mid"

#                     if left_bicep_angle > 110 and left_bicep_angle <= 140:
#                         cv2.circle(image, (lmList[13][1], lmList[13][2]), 14, (0,100,150), cv2.FILLED)
#                         left_bi_curl_stage = "Mid Down"

#                     if left_bicep_angle > 80 and left_bicep_angle <= 110:
#                         cv2.circle(image, (lmList[13][1], lmList[13][2]), 16, (0,150,100), cv2.FILLED)
#                         left_bi_curl_stage = "Mid Up"

#                     if left_bicep_angle > 50 and left_bicep_angle <= 80:
#                         cv2.circle(image, (lmList[13][1], lmList[13][2]), 18, (0,200,50), cv2.FILLED)
#                         left_bi_curl_stage = "Up Mid"

#                     if left_bicep_angle > 30 and left_bicep_angle <= 50:
#                         cv2.circle(image, (lmList[13][1], lmList[13][2]), 18, (0,225,30), cv2.FILLED)
#                         left_bi_curl_stage = "Almost There"

                    if left_shoulder > 165 and right_shoulder > 165 and left_elbow >=155 and right_elbow >= 155 and shoulder_press_stage == "DONE":
                        shoulder_press_stage = "UP"
                        cv2.circle(image, (lmList[12][1], lmList[12][2]), 20, (0,255,255), cv2.FILLED)
                        cv2.circle(image, (lmList[11][1], lmList[11][1]), 20, (0,255,255), cv2.FILLED)
                        shoulder_press_counter +=1
                        print(f"Current Should Press Rep: {shoulder_press_counter}")

                    #add curl counter
                    #setup a status box
                    cv2.rectangle(image, (0,0), (300,60), (24,117,16), -1)  #(0,0) =  start point of rectangle
                                                                            #(225,73) = end point of rectangle
                                                                            #(24,117,16) = RGB
                                                                            #-1 = fill the box with color
                    #rep data(left arm)
                    cv2.putText(image, 'SHOULDER REPS: ', (15,25),                        #(15,12) = start coord
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1, cv2.LINE_AA) #font type = hersey
                                                                                            #text size = 0.5
                                                                                            #text color = 255,255,0
                                                                                            #line thickness = 1
                                                                                            #line type = AA
                    cv2.putText(image, str(shoulder_press_counter), (10,60), #(10,60) = start coord
                                cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) #font type = hersey
                                                                                            #text size = 0.5
                                                                                            #text color = 255,255,255
                                                                                            #line thickness = 2
                                                                                            #line type = AA
                    #rep position(left arm)
                    cv2.putText(image, 'SHOULDER POS: ', (165,25),                  #(165,12) = start coord
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA) #font type = hersey
                                                                                          #text size = 0.5
                                                                                          #text color = 0,255,0
                                                                                          #line thickness = 1
                                                                                          #line type = AA
                    cv2.putText(image, shoulder_press_stage, (160,60),                         #(160,60) = start coord
                                cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) #font type = hersey
                                                                                            #text size = 1
                                                                                            #text color = 255,255,255
                                                                                            #line thickness = 2
                                                                                            #line type = AA
                                
                
        except:
            pass

        #separate window will pop open to show the video feed
        #'Live Feed' is the name of the window
        cv2.imshow('Live Workout Feed', image)

        #how to turn it off
        #the 'e' letter will exit the live feed window
        if cv2.waitKey(10) & 0xFF == ord('e'):
            break
        
#release the camera feed
cap.release()

#close the window
cv2.destroyAllWindows()

Wide_Squat [1.02471804e-05 1.74222868e-03 1.60464700e-04 4.40206680e-05
 4.56643504e-01 5.41399535e-01]
Wide_Squat [1.02136463e-05 4.88521094e-03 1.59939576e-04 1.67716212e-04
 4.55149127e-01 5.39627792e-01]
Wide_Squat [1.09578525e-05 2.78622801e-03 8.91775274e-05 1.23233846e-04
 4.56143934e-01 5.40846469e-01]
Wide_Squat [8.38591390e-06 8.22653477e-04 6.70159982e-05 1.51630849e-04
 4.89295098e-01 5.09655216e-01]
Wide_Squat [8.39290387e-06 8.23339190e-04 6.70718586e-05 1.32430722e-04
 4.88888732e-01 5.10080033e-01]
Shoulder_Press [8.19720271e-06 1.09207517e-03 1.69429478e-04 3.12529002e-05
 5.00512781e-01 4.98186265e-01]
Wide_Squat [1.35194577e-05 2.31176165e-03 1.87376074e-04 6.65091053e-05
 4.83392676e-01 5.14028157e-01]
Wide_Squat [1.38300824e-05 2.16827756e-03 1.91681249e-04 3.55767989e-05
 4.71752104e-01 5.25838531e-01]
Wide_Squat [1.21689582e-05 2.17757941e-03 2.17937576e-04 3.29865846e-05
 4.71737299e-01 5.25822029e-01]
Wide_Squat [1.01217671e-05 1.85578913e-03 2.52153895e-04 3.5

Left_Bicep_Curl [5.00522798e-01 4.09288085e-02 3.95724993e-01 5.81247240e-02
 4.41962917e-03 2.79047781e-04]
Right_Bicep_Curl [3.46933440e-01 2.08332594e-02 5.35953306e-01 9.18567757e-02
 4.16052994e-03 2.62688702e-04]
Right_Bicep_Curl [1.03913775e-01 1.37160096e-02 7.83147176e-01 9.47728513e-02
 4.18589742e-03 2.64290360e-04]
Right_Bicep_Curl [5.13936173e-02 8.34908302e-03 9.30152242e-01 8.62721997e-03
 1.39007147e-03 8.77667210e-05]
Current Right Bicep Curl Rep: 1
Left_Lat_Raise [9.96539985e-02 5.07853364e-01 2.44915061e-01 1.32897969e-02
 4.49475830e-04 1.33838304e-01]
Wide_Squat [1.66513769e-04 1.44801960e-03 1.04987979e-04 5.34207456e-06
 1.68945069e-07 9.98274968e-01]
Wide_Squat [1.96850226e-04 6.46429118e-03 4.20873935e-04 4.42026678e-06
 3.44563470e-07 9.92913220e-01]
Wide_Squat [2.40992315e-04 1.00887810e-03 4.73233586e-04 5.15155166e-06
 6.51014063e-07 9.98271093e-01]
Wide_Squat [9.85980105e-05 6.99760370e-04 1.55433579e-04 5.15552403e-06
 1.90899270e-07 9.99040862e-01]
Left_

Left_Bicep_Curl [9.99749672e-01 6.29661737e-05 8.82466230e-05 9.67918719e-05
 1.18142655e-06 1.14185838e-06]
Current Left Bicep Curl Rep: 1
Left_Bicep_Curl [9.99818545e-01 4.68815419e-05 6.68043103e-05 6.59343688e-05
 9.32921654e-07 9.01676374e-07]
Left_Bicep_Curl [9.99878198e-01 2.79197708e-05 5.14171516e-05 4.13918007e-05
 5.45591619e-07 5.27318742e-07]
Left_Bicep_Curl [9.99891559e-01 2.44116969e-05 4.38377076e-05 3.92538607e-05
 4.77038917e-07 4.61061998e-07]
Left_Bicep_Curl [9.99875912e-01 2.59646896e-05 5.23515540e-05 4.47744576e-05
 5.07386580e-07 4.90393261e-07]
Left_Bicep_Curl [9.99833552e-01 3.22621669e-05 6.35089612e-05 6.94367015e-05
 6.30448150e-07 6.09333270e-07]
Left_Bicep_Curl [9.99846307e-01 3.18605678e-05 7.35797755e-05 4.70283324e-05
 6.22600339e-07 6.01748297e-07]
Left_Bicep_Curl [9.99759988e-01 4.06476524e-05 1.47199450e-04 5.06024396e-05
 7.94312340e-07 7.67709344e-07]
Left_Bicep_Curl [9.99703996e-01 5.48099017e-05 1.78811461e-04 6.06560967e-05
 8.77833959e-07 8.48

Left_Bicep_Curl [8.85426524e-01 1.11330047e-01 2.49113751e-03 7.24092634e-04
 1.43391548e-05 1.38589098e-05]
Left_Bicep_Curl [9.34286640e-01 5.85520514e-02 5.87558660e-03 1.23752933e-03
 2.45067049e-05 2.36859298e-05]
Left_Bicep_Curl [9.55520827e-01 1.50696419e-02 2.65662224e-02 2.73673288e-03
 5.41953259e-05 5.23802237e-05]
Left_Bicep_Curl [9.97578648e-01 4.91946742e-04 1.71561907e-03 2.06273555e-04
 3.82017582e-06 3.69223103e-06]
Left_Bicep_Curl [9.92851355e-01 7.77026715e-04 6.18223869e-03 1.82725050e-04
 3.38405869e-06 3.27072027e-06]
Left_Bicep_Curl [9.76282985e-01 9.01309783e-04 2.20070327e-02 7.96762735e-04
 6.05617689e-06 5.85334427e-06]
Left_Bicep_Curl [6.67183180e-01 7.87332074e-03 3.21787180e-01 3.05228441e-03
 5.29032570e-05 5.11314287e-05]
Right_Bicep_Curl [3.10111952e-01 1.41260982e-02 6.70098968e-01 5.47632580e-03
 9.49175869e-05 9.17386206e-05]
Right_Bicep_Curl [1.22654540e-01 1.81850235e-02 8.51960549e-01 6.96257382e-03
 1.20677756e-04 1.16636033e-04]
Right_Bicep_Curl 

Left_Bicep_Curl [7.47921086e-01 2.42351788e-01 8.22320381e-03 1.46144200e-03
 2.16017190e-05 2.08782373e-05]
Left_Bicep_Curl [5.90146060e-01 3.92412144e-01 1.53199525e-02 2.06190958e-03
 3.04772897e-05 2.94565486e-05]
Left_Lat_Raise [3.31205868e-01 6.47451153e-01 1.77891170e-02 3.45347912e-03
 5.10462170e-05 4.93365843e-05]
Left_Lat_Raise [3.84105302e-01 5.80849313e-01 3.05526396e-02 4.36584251e-03
 6.45319506e-05 6.23706557e-05]
Left_Lat_Raise [1.70515427e-01 8.17477080e-01 5.73508078e-03 6.09524103e-03
 9.00943614e-05 8.70769338e-05]
Left_Lat_Raise [5.66568197e-02 9.38170126e-01 2.50447708e-03 2.58819137e-03
 4.08776102e-05 3.95085431e-05]
Left_Lat_Raise [1.19838710e-01 8.74540884e-01 2.72105708e-03 2.81201074e-03
 4.44125887e-05 4.29251286e-05]
Left_Lat_Raise [1.02881539e-01 8.90937111e-01 2.99263238e-03 3.09266368e-03
 4.88451905e-05 4.72092743e-05]
Left_Lat_Raise [1.03698346e-01 8.89848101e-01 3.26873163e-03 3.08888346e-03
 4.87854861e-05 4.71515696e-05]
Left_Lat_Raise [7.60195927

Shoulder_Press [5.96346464e-06 2.31037050e-05 1.27552349e-06 4.25118030e-06
 9.99965390e-01 1.58566837e-08]
Shoulder_Press [3.51420302e-06 3.56775211e-05 8.65328777e-07 4.90721841e-06
 9.99955022e-01 1.32880995e-08]
Shoulder_Press [2.80382959e-06 7.63349618e-05 7.54308670e-07 4.44127319e-06
 9.99915654e-01 1.15832605e-08]
Shoulder_Press [4.26774916e-06 2.52214553e-05 1.27551753e-06 8.50151744e-06
 9.99960718e-01 1.58566096e-08]
Shoulder_Press [3.91636939e-06 2.66263833e-05 1.27551747e-06 7.49726537e-06
 9.99960669e-01 1.58566088e-08]
Shoulder_Press [7.64417135e-06 9.15739329e-05 2.01649455e-06 3.72231421e-06
 9.99895027e-01 1.58555679e-08]
Shoulder_Press [7.66387079e-06 3.58921467e-05 2.80683430e-06 6.69467407e-06
 9.99946927e-01 1.58563909e-08]
Shoulder_Press [1.12373520e-05 9.58264956e-06 7.61867507e-06 8.01099598e-06
 9.99963534e-01 1.58566543e-08]
Shoulder_Press [7.48872287e-06 3.05199403e-05 2.80685661e-06 4.29470553e-06
 9.99954874e-01 1.58565169e-08]
Shoulder_Press [7.66388813e-

Shoulder_Press [5.74235972e-06 2.73695609e-05 2.07895931e-06 8.64899667e-06
 9.99956142e-01 1.85683241e-08]
Shoulder_Press [6.90428449e-06 1.61112550e-05 1.49366474e-06 9.38106159e-06
 9.99966091e-01 1.85685089e-08]
Shoulder_Press [6.29561372e-06 1.63736033e-05 1.09744308e-05 2.10671893e-06
 9.99964229e-01 2.04616128e-08]
Shoulder_Press [7.05979462e-04 1.17927722e-01 5.24529534e-04 4.15906925e-04
 8.80423810e-01 2.05138476e-06]
Wide_Squat [5.49015418e-04 2.20535626e-01 3.08055572e-04 2.44261643e-04
 1.34277258e-04 7.78228764e-01]
Wide_Squat [3.99744902e-06 4.52130449e-03 7.86900341e-06 3.82972015e-06
 2.52921141e-06 9.95460470e-01]
Wide_Squat [1.84762231e-05 4.52123482e-03 8.78868563e-06 3.84158067e-06
 2.52917243e-06 9.95445130e-01]
Wide_Squat [2.24567903e-05 3.75073497e-03 8.79542568e-06 6.77132176e-06
 2.70409308e-06 9.96208537e-01]
Wide_Squat [2.24493533e-05 4.07184665e-03 8.79251289e-06 5.70639776e-06
 2.53027382e-06 9.95888675e-01]
Wide_Squat [1.62464780e-05 5.00663741e-03 8.7844

Right_Bicep_Curl [1.93292747e-01 1.45627965e-01 6.52752670e-01 7.42421217e-03
 7.03248553e-04 1.99156051e-04]
Right_Bicep_Curl [1.68055898e-01 1.39337651e-01 6.83109557e-01 8.57759497e-03
 7.16415002e-04 2.02884716e-04]
Right_Bicep_Curl [1.84802741e-01 1.45519726e-01 6.60978406e-01 8.29970097e-03
 2.03114405e-04 1.96311726e-04]
Right_Bicep_Curl [1.84802741e-01 1.45519726e-01 6.60978406e-01 8.29970097e-03
 2.03114405e-04 1.96311726e-04]
Right_Bicep_Curl [1.48073310e-01 1.51712023e-01 6.89105002e-01 1.06932429e-02
 2.11757526e-04 2.04665373e-04]
Right_Bicep_Curl [1.32651046e-01 1.54458436e-01 7.01579736e-01 1.08868206e-02
 2.15590932e-04 2.08370391e-04]
Right_Bicep_Curl [1.31256194e-01 1.62603214e-01 6.94202486e-01 1.15186033e-02
 2.13323950e-04 2.06179335e-04]
Right_Bicep_Curl [1.27690601e-01 1.85351211e-01 6.75344381e-01 1.12056988e-02
 2.07528976e-04 2.00578445e-04]
Right_Bicep_Curl [1.41980246e-01 1.81345458e-01 6.65311484e-01 1.09635247e-02
 2.03043923e-04 1.96243605e-04]
Right_Bice

Right_Bicep_Curl [1.00562994e-02 8.06641996e-03 9.80144775e-01 1.68356908e-03
 2.48850014e-05 2.40515564e-05]
Right_Bicep_Curl [3.22105849e-02 6.72530899e-03 9.59368325e-01 1.64788192e-03
 2.43575058e-05 2.35417277e-05]
Right_Bicep_Curl [1.00848856e-01 4.98656836e-03 8.92994991e-01 1.13654813e-03
 1.67994305e-05 1.62367864e-05]
Right_Bicep_Curl [2.15095813e-01 1.95375051e-02 7.62886781e-01 2.40985363e-03
 3.56202852e-05 3.44272957e-05]
Right_Bicep_Curl [1.54241230e-01 1.72722679e-02 8.26708211e-01 1.72806148e-03
 2.55426479e-05 2.46871771e-05]
Left_Lat_Raise [3.90740669e-01 4.03896275e-01 2.02524812e-01 2.75807497e-03
 4.07673793e-05 3.94020040e-05]
Left_Lat_Raise [1.51504842e-01 7.91086386e-01 5.49877096e-02 2.35602952e-03
 3.30704858e-05 3.19628938e-05]
Left_Lat_Raise [5.54000186e-02 9.30955872e-01 1.31784528e-02 4.53148085e-04
 6.36062798e-06 6.14759873e-06]
Left_Lat_Raise [9.35194571e-02 9.01729274e-01 4.41986043e-03 3.22506030e-04
 4.52686648e-06 4.37525331e-06]
Left_Lat_Raise [8.